In [ ]:
#### ```python
# --- Imports and Setup (similar to section4.ipynb) ---
import os
import json
import asyncio
from dotenv import load_dotenv
from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

load_dotenv()

# --- Azure OpenAI Configuration ---
api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
gpt_api_version = os.getenv('AZURE_GPT_API_VERSION')
embedding_api_version = os.getenv('AZURE_EMBEDDING_API_VERSION')
llm_deployment_name = "gpt-4o" # Replace with your deployment name
embedding_deployment_name = "text-embedding-3-small" # Replace with your deployment name

# Check if environment variables are loaded
if not all([api_key, azure_endpoint, gpt_api_version, embedding_api_version]):
    raise ValueError("Azure OpenAI environment variables not set correctly.")

llm = AzureOpenAI(
    model="gpt-4o",
    deployment_name=llm_deployment_name,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=gpt_api_version,
)
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    deployment_name=embedding_deployment_name,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=embedding_api_version,
)

Settings.llm = llm
Settings.embed_model = embed_model

print("LLM and Embedding models initialized.")

# --- Placeholder Workflow Functions (Tools for the Supervisor) ---
# These functions simulate running the workflows from section1-4.
# They expect an input string like "Submission: {...} State: {...}"
# They return a JSON string like {"status": "...", "reason": "...", "new_state": {...}}

async def run_missing_value_check(input_str: str) -> str:
    """
    Simulates running the missing value check workflow (section1.ipynb).
    Parses input string for submission and state. Checks if required fields exist.
    Updates state and returns JSON result.
    """
    print(f"\n--- Running Missing Value Check ---")
    try:
        submission_str = input_str.split(" State: ")[0].replace("Submission: ", "")
        state_str = input_str.split(" State: ")[1]
        submission_details = json.loads(submission_str)
        current_state = json.loads(state_str)
        print(f"  Input State: {current_state}")
    except Exception as e:
        print(f"  Error parsing input: {e}")
        return json.dumps({"status": "error", "reason": f"Input parsing error: {e}", "new_state": {}})

    if current_state.get("missing_value_passed", False):
        print("  Check previously passed. Skipping.")
        return json.dumps({"status": "skipped", "reason": "Previously passed", "new_state": current_state})

    # Simulate check: Check if 'client' exists
    if submission_details.get("client"):
        print("  Check PASSED.")
        current_state["missing_value_passed"] = True
        return json.dumps({"status": "passed", "reason": "All required values present.", "new_state": current_state})
    else:
        print("  Check FAILED: 'client' field missing.")
        current_state["missing_value_passed"] = False
        return json.dumps({"status": "failed", "reason": "'client' field missing", "new_state": current_state})

async def run_duplication_check(input_str: str) -> str:
    """
    Simulates running the duplication check workflow (section2.ipynb).
    Parses input, checks state, simulates logic, updates state, returns JSON.
    """
    print(f"\n--- Running Duplication Check ---")
    try:
        state_str = input_str.split(" State: ")[1]
        current_state = json.loads(state_str)
        print(f"  Input State: {current_state}")
    except Exception as e:
        print(f"  Error parsing input: {e}")
        return json.dumps({"status": "error", "reason": f"Input parsing error: {e}", "new_state": current_state})

    if current_state.get("duplication_check_passed", False):
        print("  Check previously passed. Skipping.")
        return json.dumps({"status": "skipped", "reason": "Previously passed", "new_state": current_state})

    # Simulate check: Always passes in this example
    print("  Check PASSED (Simulation).")
    current_state["duplication_check_passed"] = True
    return json.dumps({"status": "passed", "reason": "No duplicate found.", "new_state": current_state})


async def run_compliance_check(input_str: str) -> str:
    """
    Simulates running the compliance check workflow (section3.ipynb).
    Parses input, checks state, simulates logic (e.g., coverage amount), updates state, returns JSON.
    """
    print(f"\n--- Running Compliance Check ---")
    try:
        submission_str = input_str.split(" State: ")[0].replace("Submission: ", "")
        state_str = input_str.split(" State: ")[1]
        submission_details = json.loads(submission_str)
        current_state = json.loads(state_str)
        print(f"  Input State: {current_state}")
    except Exception as e:
        print(f"  Error parsing input: {e}")
        return json.dumps({"status": "error", "reason": f"Input parsing error: {e}", "new_state": current_state})

    if current_state.get("compliance_check_passed", False):
        print("  Check previously passed. Skipping.")
        return json.dumps({"status": "skipped", "reason": "Previously passed", "new_state": current_state})

    # Simulate check: Fails if coverage > 800B IRR, passes otherwise or on resubmission
    coverage_str = submission_details.get("coverage_requirements", {}).get("desired_coverage_amount", "IRR 0")
    try:
        # Handle potential formatting like "IRR 800,000,000,000"
        coverage = int(''.join(filter(str.isdigit, coverage_str)))
    except ValueError:
        print("  Error parsing coverage amount.")
        return json.dumps({"status": "error", "reason": "Invalid coverage amount format", "new_state": current_state})

    limit = 800_000_000_000
    is_resubmission = current_state.get("resubmission", False)

    if coverage <= limit:
        print(f"  Check PASSED (Coverage: {coverage} <= {limit}).")
        current_state["compliance_check_passed"] = True
        current_state["compliance_issues"] = []
        return json.dumps({"status": "passed", "reason": "Coverage within limits.", "new_state": current_state})
    else: # Coverage > limit
        print(f"  Check FAILED (Coverage: {coverage} > {limit}).")
        current_state["compliance_check_passed"] = False
        current_state["compliance_issues"] = ["Coverage amount exceeds limit"]
        return json.dumps({"status": "failed", "reason": f"Coverage amount {coverage} exceeds limit {limit}", "new_state": current_state})


async def run_guideline_check(input_str: str) -> str:
    """
    Simulates running the guideline check workflow (section4.ipynb).
    Parses input, checks state (requires compliance pass), simulates logic, updates state, returns JSON.
    """
    print(f"\n--- Running Guideline Check ---")
    try:
        state_str = input_str.split(" State: ")[1]
        current_state = json.loads(state_str)
        print(f"  Input State: {current_state}")
    except Exception as e:
        print(f"  Error parsing input: {e}")
        return json.dumps({"status": "error", "reason": f"Input parsing error: {e}", "new_state": current_state})

    if current_state.get("guideline_check_passed", False):
        print("  Check previously passed. Skipping.")
        return json.dumps({"status": "skipped", "reason": "Previously passed", "new_state": current_state})

    # Guideline check requires prior checks to pass
    if not current_state.get("missing_value_passed", False) or \
       not current_state.get("duplication_check_passed", False) or \
       not current_state.get("compliance_check_passed", False):
        print("  Check SKIPPED (Due to prior check failure).")
        current_state["guideline_check_passed"] = False # Mark as not passed
        return json.dumps({"status": "skipped", "reason": "Skipped due to prior check failure", "new_state": current_state})
    else:
        # Simulate guideline check (assume it passes if compliance passed)
        print("  Check PASSED (Simulation).")
        current_state["guideline_check_passed"] = True
        current_state["guideline_result"] = "All guidelines met."
        # In reality, this might trigger the email agent from section4 if issues were found
        return json.dumps({"status": "passed", "reason": "Submission meets guidelines.", "new_state": current_state})

# --- Create Tools ---
tools = [
    FunctionTool.from_defaults(fn=run_missing_value_check, name="run_missing_value_check"),
    FunctionTool.from_defaults(fn=run_duplication_check, name="run_duplication_check"),
    FunctionTool.from_defaults(fn=run_compliance_check, name="run_compliance_check"),
    FunctionTool.from_defaults(fn=run_guideline_check, name="run_guideline_check"),
]

# --- Supervisory Agent Definition ---
supervisor_system_prompt = """
You are a supervisory agent orchestrating insurance quote checks.
Your goal is to decide which checking workflows (tools) to run based on the submission and its current state provided in the input.

Available tools: run_missing_value_check, run_duplication_check, run_compliance_check, run_guideline_check.
Each tool takes a single string argument: 'Submission: {submission_json} State: {current_state_json}'.
Each tool returns a JSON string: {"status": "...", "reason": "...", "new_state": {...}}. You MUST parse this JSON result to get the 'new_state'.

Workflow Logic:
1.  **Parse Input:** Extract the submission details (JSON) and current state (JSON) from the user prompt.
2.  **Determine Starting Point:** Check the 'current_state'.
    - If it's a new submission (e.g., empty state or no checks passed), start with `run_missing_value_check`.
    - If it's a resubmission (e.g., `resubmission: true` in state) or some checks previously passed/failed, identify the FIRST check that hasn't passed (`missing_value_passed`, `duplication_check_passed`, `compliance_check_passed`, `guideline_check_passed` are all false or missing for that check and subsequent ones). Start processing from that check. Do NOT re-run checks marked as passed in the state.
3.  **Execute Sequentially:** Run the necessary checks in the standard order: missing value -> duplication -> compliance -> guideline.
4.  **State Update:** Before calling the *next* tool, YOU MUST update the `current_state_json` part of the tool input string using the `new_state` returned by the *previous* tool call.
5.  **Stop on Failure:** If any check tool returns a status of "failed" or "error", stop the sequence immediately.
6.  **Final Output:** Summarize the outcome. State which checks were run and the final status (e.g., "All checks passed.", "Compliance check failed: [reason from tool output]."). Include the final state JSON in your response.
"""

supervisor_agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True, # Set to True to see the agent's reasoning steps
    system_prompt=supervisor_system_prompt,
)

# --- Simulation Runner ---
async def run_scenario(scenario_name: str, submission_dict: dict, initial_state_dict: dict):
    print(f"\n{'='*20} SCENARIO: {scenario_name} {'='*20}")
    input_prompt = f"Process this submission. Submission: {json.dumps(submission_dict)} State: {json.dumps(initial_state_dict)}"
    print(f"Input Prompt:\n{input_prompt}\n")
    response = await supervisor_agent.achat(input_prompt)
    print(f"\nFinal Supervisor Response ({scenario_name}):\n{response}\n")
    return response

# --- Define Scenarios ---

# Scenario 1: New Submission (expected to fail compliance)
new_submission_dict = {
  "insurance_broker": "Prime Insurance Brokers", "date": "25 May 2025", "client": "Parsian Evin Hotel Ltd.",
  "coverage_requirements": { "desired_coverage_amount": "IRR 950,000,000,000" } # > 800B limit
}
initial_state_dict_s1 = {}

# Scenario 2: Resubmission after Compliance Failure (expected to pass all)
# State reflects outcome of a previous run similar to Scenario 1
previous_state_dict_s2 = {
    "missing_value_passed": True,
    "duplication_check_passed": True,
    "compliance_check_passed": False, # Failed previously
    "compliance_issues": ["Coverage amount exceeds limit"],
    "guideline_check_passed": None, # Not reached
    "resubmission": True # Flag for supervisor logic
}
corrected_submission_dict = {
  "insurance_broker": "Prime Insurance Brokers", "date": "25 May 2025", "client": "Parsian Evin Hotel Ltd.",
  "coverage_requirements": { "desired_coverage_amount": "IRR 500,000,000,000" } # Corrected amount <= 800B limit
}

# Scenario 3: New Submission (expected to pass all)
pass_submission_dict = {
  "insurance_broker": "Prime Insurance Brokers", "date": "25 May 2025", "client": "Parsian Evin Hotel Ltd.",
  "coverage_requirements": { "desired_coverage_amount": "IRR 100,000,000,000" } # <= 800B limit
}
initial_state_dict_s3 = {}


# --- Run Simulations Asynchronously ---
async def main():
    await run_scenario("New Submission - Fails Compliance", new_submission_dict, initial_state_dict_s1)
    await run_scenario("Resubmission - Corrected Compliance", corrected_submission_dict, previous_state_dict_s2)
    await run_scenario("New Submission - Passes All", pass_submission_dict, initial_state_dict_s3)

# Execute the main async function
# In a Jupyter notebook, you might need to use await main() directly if running in an async environment
# or asyncio.run(main()) if not.
# await main() # Use this if running in Jupyter/IPython with top-level await enabled
asyncio.run(main()) # Use this in a standard Python script